In [1]:
import os

os.environ["HF_HOME"] = "/NS/llm-1/nobackup/afkhan/HF_CACHE/Misc"
os.environ["HF_DATASETS_CACHE"] = "/NS/llm-1/nobackup/afkhan/HF_CACHE/Datasets"
os.environ["TRANSFORMERS_CACHE"] = "/NS/llm-1/nobackup/afkhan/HF_CACHE/Models"

cache_dir = os.getenv("TRANSFORMERS_CACHE")

In [2]:
import pandas as pd

In [3]:
path_template = '/NS/llm-1/work/afkhan/Perplexity-vs-Evaluation/Experiments/LMEvalHarness_Runs/benchmarks_samples/hellaswag_okapi_template_aya_23_8B/samples_hellaswag_hi_2024-08-04T22-30-16.191215.jsonl.feather'
path_no_template = '/NS/llm-1/work/afkhan/Perplexity-vs-Evaluation/Experiments/LMEvalHarness_Runs/benchmarks_samples/hellaswag_okapi_no_template_aya_23_8B/samples_hellaswag_hi_2024-08-04T14-52-52.186900.jsonl.feather'

df_template = pd.read_feather(path_template)
df_no_template = pd.read_feather(path_no_template)

In [4]:
strings_to_process_template = df_template['gen_args_0.arg_0'].to_list()
strings_to_process_no_template = df_no_template['gen_args_0.arg_0'].to_list()

In [5]:
model_path = '/NS/llm-1/nobackup/afkhan/Model_Saves/aya-23-8B'

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [7]:
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
device = 'cuda:0'

In [9]:
model.to(device)
tokenizer.to(device)

CohereForCausalLM(
  (model): CohereModel(
    (embed_tokens): Embedding(256000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x CohereDecoderLayer(
        (self_attn): CohereSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): CohereRotaryEmbedding()
        )
        (mlp): CohereMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): CohereLayerNorm()
      )
    )
    (norm): CohereLayerNorm()
  )
  (lm_head): Linear(in_features=409

In [22]:
def get_perplexity_on_string(string, model, tokenizer):
    inputs = tokenizer(string, return_tensors='pt')
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        loss = model(**inputs, labels=inputs['input_ids']).loss
    return torch.exp(loss).item()

# No Template

In [23]:
sample_no_template = strings_to_process_no_template[:10]

results_no_template = []

for sample in sample_no_template:
    loss = get_loss_on_string(sample, model, tokenizer)
    results_no_template.append(loss)

for input_text, result in zip(sample_no_template, results_no_template):
    print(f"Input text: {input_text}")
    print(f"Perplexity: {result}")
    print("")

Input text: Personal Care and Style: धनुष से अपनी फैशन वार्ड्रोब को कैसे अपडेट करें. ऐसी ब्लाउज खोजें जिनके कम से कम एक धनुष कहीं पर हो। यह टॉप के लिए सबसे आसान विकल्प है क्योंकि आपको अपने टॉप के रंग और साइज़ से मिलती-जुलती धनुष बनाने की चिंता नहीं करनी पड़ती।. धनुष मुद्रित ब्लाउज को एक विकल्प के रूप में विचार करें।
Perplexity: 1.548416256904602

Input text: Health: अपने बच्चे को दांत पिसने से कैसे रोकें. समझें कि तनाव दांत पिसने का कारण हो सकता है। दांत पिसाई, आपके बच्चे के स्ट्रेस या कुछ चिंता के बारे में परेशान होने का संकेत हो सकता है। अपने बच्चे को आराम दिलाने में मदद करना आपके बच्चे को दांत पिसने से रोकने की कुंजी हो सकता है।
Perplexity: 1.1903691291809082

Input text: Pets and Animals: अपने बच्चों को कुत्ता चुनने में शामिल करने के लिए कैसे जोड़ें. एक कुत्ते को पाने की संभावना लाओ। अधिकांश बच्चों के लिए कुत्ता पाने का विचार बहुत उत्साहजनक होगा। अपने बच्चों के साथ इस मुद्दे को उठाएं जब वे उत्साहित होने और प्रश्न पूछने के लिए समय हो।
Perplexity: 1.2255840301513672

Input text: High

# With Template

In [24]:
sample_template = strings_to_process_template[:10]

results_template = []

for sample in sample_template:
    loss = get_loss_on_string(sample, model, tokenizer)
    results_template.append(loss)

for input_text, result in zip(sample_template, results_template):
    print(f"Input text: {input_text}")
    print(f"Perplexity: {result}")
    print("")

Input text: <BOS_TOKEN><|START_OF_TURN_TOKEN|><|USER_TOKEN|>Personal Care and Style: धनुष से अपनी फैशन वार्ड्रोब को कैसे अपडेट करें. ऐसी ब्लाउज खोजें जिनके कम से कम एक धनुष कहीं पर हो। यह टॉप के लिए सबसे आसान विकल्प है क्योंकि आपको अपने टॉप के रंग और साइज़ से मिलती-जुलती धनुष बनाने की चिंता नहीं करनी पड़ती।. धनुष मुद्रित ब्लाउज को एक विकल्प के रूप में विचार करें।<|END_OF_TURN_TOKEN|><|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>
Perplexity: 1.9173524379730225

Input text: <BOS_TOKEN><|START_OF_TURN_TOKEN|><|USER_TOKEN|>Health: अपने बच्चे को दांत पिसने से कैसे रोकें. समझें कि तनाव दांत पिसने का कारण हो सकता है। दांत पिसाई, आपके बच्चे के स्ट्रेस या कुछ चिंता के बारे में परेशान होने का संकेत हो सकता है। अपने बच्चे को आराम दिलाने में मदद करना आपके बच्चे को दांत पिसने से रोकने की कुंजी हो सकता है।<|END_OF_TURN_TOKEN|><|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>
Perplexity: 1.6922978162765503

Input text: <BOS_TOKEN><|START_OF_TURN_TOKEN|><|USER_TOKEN|>Pets and Animals: अपने बच्चों को कुत्ता चुनने में

# Comparing

In [26]:
for input_text_no_template, input_text_template, result_no_template, result_template in zip(sample_no_template, sample_template, results_no_template, results_template):
    print(f"Input text (no template): {input_text_no_template}")
    print(f"Input text (template): {input_text_template}")
    print(f"Perplexity (no template): {result_no_template}")
    print(f"Perplexity (template): {result_template}")
    print("")

Input text (no template): Personal Care and Style: धनुष से अपनी फैशन वार्ड्रोब को कैसे अपडेट करें. ऐसी ब्लाउज खोजें जिनके कम से कम एक धनुष कहीं पर हो। यह टॉप के लिए सबसे आसान विकल्प है क्योंकि आपको अपने टॉप के रंग और साइज़ से मिलती-जुलती धनुष बनाने की चिंता नहीं करनी पड़ती।. धनुष मुद्रित ब्लाउज को एक विकल्प के रूप में विचार करें।
Input text (template): <BOS_TOKEN><|START_OF_TURN_TOKEN|><|USER_TOKEN|>Personal Care and Style: धनुष से अपनी फैशन वार्ड्रोब को कैसे अपडेट करें. ऐसी ब्लाउज खोजें जिनके कम से कम एक धनुष कहीं पर हो। यह टॉप के लिए सबसे आसान विकल्प है क्योंकि आपको अपने टॉप के रंग और साइज़ से मिलती-जुलती धनुष बनाने की चिंता नहीं करनी पड़ती।. धनुष मुद्रित ब्लाउज को एक विकल्प के रूप में विचार करें।<|END_OF_TURN_TOKEN|><|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>
Perplexity (no template): 1.548416256904602
Perplexity (template): 1.9173524379730225

Input text (no template): Health: अपने बच्चे को दांत पिसने से कैसे रोकें. समझें कि तनाव दांत पिसने का कारण हो सकता है। दांत पिसाई, आपके बच्चे क

# Other Strings

In [27]:
input_texts = ["lorem ipsum", "Happy Birthday!", "Bienvenue"]
results = []
for input_text in input_texts:
    results.append(get_perplexity_on_string(input_text, model, tokenizer))

for input_text, result in zip(input_texts, results):
    print(f"Input text: {input_text}")
    print(f"Perplexity: {result}")
    print("")

Input text: lorem ipsum
Perplexity: 150.28932189941406

Input text: Happy Birthday!
Perplexity: 189.37318420410156

Input text: Bienvenue
Perplexity: 176.8068389892578



In [29]:
input_texts = ["git pull origin master fetches and merges the contents of the master branch with your branch and creates a merge commit. If there are any merge conflicts you'll be notified at this stage and you must resolve the merge commits before proceeding. When you are ready to push your local commits, including your new merge commit, to the remote server, run git push.",
                "Some evaluation modules require some external data such as NLTK that requires resources or the BLEURT metric that requires checkpoints. You can implement these downloads in EvaluationModule._download_and_prepare(), which downloads and caches the resources via the dlmanager. A simplified example on how BLEURT downloads and loads a checkpoint", 
                "Born in the German Empire, Einstein moved to Switzerland in 1895, forsaking his German citizenship (as a subject of the Kingdom of Württemberg)[note 1] the following year. In 1897, at the age of seventeen, he enrolled in the mathematics and physics teaching diploma program at the Swiss federal polytechnic school in Zürich, graduating in 1900. In 1901, he acquired Swiss citizenship, which he kept for the rest of his life."]
results = []

for input_text in input_texts:
    results.append(get_perplexity_on_string(input_text, model, tokenizer))

for input_text, result in zip(input_texts, results):
    print(f"Input text: {input_text}")
    print(f"Perplexity: {result}")
    print("")

Input text: git pull origin master fetches and merges the contents of the master branch with your branch and creates a merge commit. If there are any merge conflicts you'll be notified at this stage and you must resolve the merge commits before proceeding. When you are ready to push your local commits, including your new merge commit, to the remote server, run git push.
Perplexity: 11.2291259765625

Input text: Some evaluation modules require some external data such as NLTK that requires resources or the BLEURT metric that requires checkpoints. You can implement these downloads in EvaluationModule._download_and_prepare(), which downloads and caches the resources via the dlmanager. A simplified example on how BLEURT downloads and loads a checkpoint
Perplexity: 68.48452758789062

Input text: Born in the German Empire, Einstein moved to Switzerland in 1895, forsaking his German citizenship (as a subject of the Kingdom of Württemberg)[note 1] the following year. In 1897, at the age of seve